# PROGRAMA DE INTELIGENCIA ARTIFICIAL | IBM SkillUp 2024

<!-- Tabla con tres logos -->
<table align="center">
    <tr>
        <td><img src="https://thepearlhighschool.org/wp-content/uploads/2023/07/rsw_1280-1024x767.webp" alt="IBM" width="350"></td>
        <td><img src="https://uptec.up.pt/wp-content/uploads/2022/04/SkillUp_x2.png" alt="Skillup" width="350"></td>
        <td><img src="https://visionyvalor.es/wp-content/uploads/2024/03/Python-Symbol_0-3.png" alt="Python" width="300"></td>
    </tr>
</table>
<br><br>

## Introducción a la práctica: creación de un Chatbot Inteligente con Voz

### Objetivo
- En esta práctica, aprenderemos a construir un chatbot avanzado que puede interactuar con los usuarios mediante voz y texto. Utilizaremos diversas tecnologías para lograr un chatbot funcional.

#### Los objetivos específicos de esta práctica son:

- Integrar la conversión de voz a texto utilizando IBM Watson Speech to Text, para que el chatbot pueda entender las entradas de voz de los usuarios.
- Desarrollar un modelo de chatbot inteligente utilizando OpenAI GPT-3, para que el chatbot pueda entender las consultas de los usuarios y generar respuestas coherentes y naturales.
- Integrar la conversión de texto a voz utilizando IBM Watson Text to Speech, para que el chatbot pueda responder a los usuarios con salida de audio.
- Crear una interfaz web interactiva utilizando Flask, HTML, CSS y JavaScript, permitiendo a los usuarios interactuar con el chatbot de manera intuitiva.

  
#### Tecnologías que usaremos:
- Python y Flask: Para desarrollar el backend de la aplicación web.
- IBM Watson Speech to Text: Para convertir la voz del usuario en texto.
- OpenAI GPT-3.5: Para generar respuestas inteligentes a partir del texto de entrada.
- IBM Watson Text to Speech: Para convertir las respuestas de texto del chatbot en audio.
- HTML, CSS y JavaScript: Para crear la interfaz web que los usuarios utilizarán para interactuar con el chatbot.

### Descripción del Proyecto

El proyecto consiste en un chatbot que puede recibir entradas de voz y texto del usuario, procesar esas entradas para generar una respuesta inteligente, y devolver la respuesta al usuario tanto en texto como en audio. La aplicación tendrá la siguiente funcionalidad:

- Entrada de Voz: Los usuarios pueden hablarle al chatbot y el audio será convertido a texto utilizando IBM Watson Speech to Text.
- Procesamiento de Texto: El texto generado será enviado a un modelo GPT-3 de OpenAI, que procesará la entrada y generará una respuesta adecuada.
- Salida de Voz: La respuesta de texto del chatbot será convertida a audio utilizando IBM Watson Text to Speech y reproducida para el usuario.
- Interfaz Web: La aplicación web permitirá a los usuarios interactuar con el chatbot de manera fácil e intuitiva. La interfaz incluirá campos para entrada de texto y botones para grabar y enviar audio.


#### Estructura del Proyecto

- ***Backend (Python y Flask)***:
    - Configuración del servidor Flask.
    - Rutas para manejar las peticiones de transcripción de audio, generación de respuestas y conversión de texto a audio.
    - Integración con las APIs de IBM Watson y OpenAI.
    - Frontend (HTML, CSS y JavaScript):

- ***Diseño de la interfaz de usuario utilizando HTML y CSS.***
    - Implementación de la lógica de interacción utilizando JavaScript para manejar la grabación de audio, envío de peticiones al servidor y reproducción de respuestas en audio.

#### Pasos para Realizar la Práctica
- ***Configuración del Entorno:***
    - Instalación de las bibliotecas necesarias.
    - Configuración de las credenciales para IBM Watson y OpenAI.

- ***Desarrollo del Backend:***
    - Creación del servidor Flask.
    - Implementación de las rutas necesarias para manejar las funcionalidades del chatbot.

- ***Desarrollo del Frontend:***
    - Diseño de la página web para la interacción con el chatbot.
    - Implementación de la lógica para manejar las entradas de voz y texto.

- ***Fase de pruebas:***
    - Pruebas funcionales del chatbot.

### Recursos
- Documentación de IBM Watson: Speech to Text, Text to Speech
    - https://www.ibm.com/mx-es/products/speech-to-text
    - https://www.ibm.com/es-es/products/text-to-speech
- Documentación de OpenAI GPT-3: OpenAI API
    - https://platform.openai.com/docs/api-reference/authentication
- Flask Documentation: Flask
    - https://flask-es.readthedocs.io/

#### Paso 1: configuración del entorno
- Instalamos las librerías de Flask, Openai e IBM-Watson 

In [2]:
!pip install flask
!pip install ibm-watson
!pip install openai
!pip install python-dotenv #cargar variables entorno (encapsular API)

#### Paso 2: configurar el entorno Flask (Backend)
- Creamos el archivo chatbot.py
- Importamos la librerías necesarias
- Creamos el código que nos permitirá acceder a las API´s

In [4]:
%%writefile chatbot.py

# Importar las bibliotecas necesarias
from flask import Flask, render_template, request, jsonify, send_file, render_template_string
from ibm_watson import SpeechToTextV1, TextToSpeechV1
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
import openai
from dotenv import load_dotenv
from io import BytesIO  # Importación de BytesIO para manejar datos binarios en memoria
import os

# Cargar variables de entorno (claves API) desde el archivo .env
load_dotenv('claves.env')

app = Flask(__name__)

# Credenciales de IBM Watson Speech to Text
stt_authenticator = IAMAuthenticator(os.getenv('SPEECH_TO_TEXT_APIKEY'))
speech_to_text = SpeechToTextV1(authenticator=stt_authenticator)
speech_to_text.set_service_url(os.getenv('SPEECH_TO_TEXT_URL'))

# Credenciales de IBM Watson Text to Speech
tts_authenticator = IAMAuthenticator(os.getenv('TEXT_TO_SPEECH_APIKEY'))
text_to_speech = TextToSpeechV1(authenticator=tts_authenticator)
text_to_speech.set_service_url(os.getenv('TEXT_TO_SPEECH_URL'))

# Credenciales de OpenAI
#client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

openai.api_key = os.getenv('OPENAI_API_KEY')

@app.route('/')
def index():
    # Ruta para la página principal
    # Renderiza el archivo index.html ubicado en el directorio templates
    return render_template('index.html')

@app.route('/transcribe', methods=['POST'])
def transcribe():
    # ruta para manejar la transcripcion de audio a texto
    # Recibe un archivo de audio a través de una solicitud POST
    audio = request.files['audio']
    # Utiliza el servicio Speech to Text de IBM Watson para reconocer el audio
    stt_result = speech_to_text.recognize(
        audio=audio, # El archivo de audio recibido
        content_type='audio/wav', # Tipo de contenido del archivo de audio
        model='es-ES_BroadbandModel' # Modelo de idioma utilizado para la transcripción
    ).get_result()

    # Extrae el texto transcrito del resultado de la API
    transcript = stt_result['results'][0]['alternatives'][0]['transcript']

    # Devuelve el texto transcrito en formato JSON
    return jsonify({'transcript': transcript})

@app.route('/chat', methods=['POST'])
def chat():
    # Ruta para manejar las solicitudes de chat
    # Recibe un mensaje de usuario a través de una solicitud POST
    user_input = request.json['message']

    # Mensaje de depuración para verificar la entrada del usuario
    print("Mensaje del usuario:", user_input)

    # Utiliza el servicio de OpenAI para generar una respuesta
    try:
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",  # Modelo de GPT-3.5 Turbo utilizado para generar la respuesta
            messages=[{"role": "user", "content": user_input}]  # Mensaje del usuario en formato de chat
        )

        # Mensaje de depuración para verificar la respuesta de OpenAI
        print("Respuesta de OpenAI:", response.choices[0].message['content'].strip())

        # Devuelve la respuesta generada en formato JSON
        return jsonify({'response': response.choices[0].message['content'].strip()})
    except Exception as e:
        print("Error al generar la respuesta de OpenAI:", e)
        return jsonify({'response': 'Lo siento, no puedo procesar tu solicitud en este momento.'})

@app.route('/speak', methods=['POST'])
def speak():
    # Ruta para manejar la conversión de texto a voz
    # Recibe un texto a través de una solicitud POST
    text = request.json['text']
    
    # Mensaje de depuración para verificar el texto recibido
    print("Texto recibido para sintetizar:", text)
    
    # Utiliza el servicio Text to Speech de IBM Watson para sintetizar el texto
    try:
        tts_result = text_to_speech.synthesize(
            text,  # El texto recibido
            voice='es-ES_LauraV3Voice',  # Voz utilizada para la síntesis
            accept='audio/wav'  # Tipo de contenido de la respuesta (archivo de audio OGG) 
        ).get_result()
        
        # Extrae el contenido de audio de la respuesta de la API
        audio = tts_result.content

        # Mensaje de depuración para verificar la longitud del audio
        print("Audio generado, longitud:", len(audio))
        
        # Devuelve el contenido de audio como respuesta
        return send_file(BytesIO(audio), mimetype='audio/wav')
    except Exception as e:
        print("Error al generar el audio:", e)
        return jsonify({'response': 'Lo siento, no puedo procesar tu solicitud en este momento.'})

if __name__ == '__main__':
    # Inicia la aplicación Flask en modo de depuración
    app.run(debug=True)



Overwriting chatbot.py


#### Paso 3: Crear la interfaz de usuario en HTML, CSS y JavaScript (Frontend)
- Creamos el archivo index.html:
- Creamos el fronted (parte visual) que nos permitirá ejecutar nuestra app


In [6]:
import os
# Crear el directorio templates si no existe
os.makedirs("templates", exist_ok=True)

# Crea y escribe el archivo index.html dentro del directorio templates
with open("templates/index.html", "w") as file:
    file.write("""
<!DOCTYPE html>
<html lang="es">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Chatbot con voz</title>
    <style>
        /* Estilos generales para el cuerpo del documento */
        body {
            font-family: Arial, sans-serif;
            margin: 0;
            padding: 0;
            display: flex;
            justify-content: center;
            align-items: center;
            height: 100vh;
            background-color: #f0f0f0;
        }
        /* Contenedor principal */
        #container {
            text-align: center;
            background: white;
            padding: 20px;
            box-shadow: 0 0 10px rgba(0, 0, 0, 0.1);
            border-radius: 10px;
        }
        /* Caja de chat */
        #chat {
            border: 1px solid #ccc;
            padding: 10px;
            height: 300px;
            width: 400px;
            overflow-y: auto;
            margin-bottom: 10px;
            background-color: #fff;
            border-radius: 5px;
        }
        /* Campo de entrada de texto */
        #input {
            width: calc(100% - 20px);
            padding: 10px;
            margin-bottom: 10px;
            border: 1px solid #ccc;
            border-radius: 5px;
        }
        /* Botones de enviar y hablar */
        #send, #voice {
            padding: 10px 20px;
            margin: 5px;
            border: none;
            border-radius: 5px;
            background-color: #007BFF;
            color: white;
            cursor: pointer;
        }
        /* Estilo hover para los botones */
        #send:hover, #voice:hover {
            background-color: #0056b3;
        }
        /* Control de audio */
        #audio {
            margin-top: 10px;
            width: 100%;
        }
        /* Indicador de grabación */
        #recording-indicator {
            color: red;
            display: none;
        }
    </style>
</head>
<body>
    <div id="container">
        <!-- Contenedor del chat y controles -->
        <div id="chat"></div>
        <input type="text" id="input" placeholder="Escribe tu mensaje">
        <button id="send">Enviar</button>
        <button id="voice">Hablar</button>
        <p id="recording-indicator">Grabando...</p>
        <audio id="audio" style="display: none;"></audio>
    </div>

    <script>
        // Referencias a los elementos del DOM
        const chat = document.getElementById('chat');
        const input = document.getElementById('input');
        const send = document.getElementById('send');
        const voice = document.getElementById('voice');
        const audio = document.getElementById('audio');
        const recordingIndicator = document.getElementById('recording-indicator');

        // Evento click para el botón de enviar
        send.addEventListener('click', () => {
            const message = input.value;
            if (message.trim() === '') return; // No enviar mensajes vacíos
            input.value = '';
            chat.innerHTML += `<p><strong>Tú:</strong> ${message}</p>`;
            chat.scrollTop = chat.scrollHeight; // Desplazar hacia abajo
            fetch('/chat', {
                method: 'POST',
                headers: { 'Content-Type': 'application/json' },
                body: JSON.stringify({ message: message })
            })
            .then(response => response.json())
            .then(data => {
                chat.innerHTML += `<p><strong>Bot:</strong> ${data.response}</p>`;
                chat.scrollTop = chat.scrollHeight; // Desplazar hacia abajo
                fetch('/speak', {
                    method: 'POST',
                    headers: { 'Content-Type': 'application/json' },
                    body: JSON.stringify({ text: data.response })
                })
                .then(response => response.blob())
                .then(blob => {
                    const url = URL.createObjectURL(blob);
                    audio.src = url;
                    audio.play(); // Reproducir el audio
                });
            });
        });

        // Evento click para el botón de hablar
        voice.addEventListener('click', () => {
            const recognition = new (window.SpeechRecognition || window.webkitSpeechRecognition)();
            recognition.lang = 'es-ES'; // Idioma de reconocimiento
            recognition.start();
            recordingIndicator.style.display = 'block'; // Mostrar indicador de grabación
            recognition.onresult = (event) => {
                const transcript = event.results[0][0].transcript;
                input.value = transcript;
                send.click(); // Simular el click en el botón de enviar
            };
            recognition.onend = () => {
                recordingIndicator.style.display = 'none'; // Ocultar indicador de grabación
            };
        });
    </script>
</body>
</html>
""")



#### Paso 4: Ejecutar la aplicación
- Ejecuta el chatbot Flask desde Jupyter Notebooks:

In [ ]:
!python chatbot.py


 * Serving Flask app 'chatbot'
 * Debug mode: on
 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with watchdog (fsevents)
 * Debugger is active!
 * Debugger PIN: 332-424-502
127.0.0.1 - - [16/Jul/2024 06:35:12] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [16/Jul/2024 06:35:12] "GET /?__debugger__=yes&cmd=resource&f=debugger.js HTTP/1.1" 304 -
127.0.0.1 - - [16/Jul/2024 06:35:12] "GET /?__debugger__=yes&cmd=resource&f=console.png HTTP/1.1" 304 -
127.0.0.1 - - [16/Jul/2024 06:35:12] "GET /?__debugger__=yes&cmd=resource&f=style.css HTTP/1.1" 304 -
Mensaje del usuario: Hola
Respuesta de OpenAI: ¡Hola! ¿Cómo puedo ayudarte hoy?
127.0.0.1 - - [16/Jul/2024 06:35:17] "POST /chat HTTP/1.1" 200 -
Texto recibido para sintetizar: ¡Hola! ¿Cómo puedo ayudarte hoy?
Audio generado, longitud: 118438
127.0.0.1 - - [16/Jul/2024 06:35:18] "POST /speak HTTP/1.1" 200 -
Mensaje del usuario: Cómo te encuentras
Respuesta de OpenAI: ¡Hola! Soy un asistente de inteligencia artificial, así que no 